In [1]:
%cd ..

/kaggle/working


In [2]:
from hydra import compose, initialize
from omegaconf import OmegaConf

with initialize(version_base=None, config_path="../cand_unsupervised/image_sim"):
    cfg = compose(config_name="config.yaml", overrides=["debug=True"])
    print(OmegaConf.to_yaml(cfg))

debug: true
seed: 7
dir:
  data_dir: /kaggle/working/input/atmaCup16_Dataset
  output_dir: /kaggle/working/output
  exp_dir: /kaggle/working/output/exp
  cand_unsupervised_dir: /kaggle/working/output/cand_unsupervised
  cand_supervised_dir: /kaggle/working/output/cand_supervised
  datasets_dir: /kaggle/working/output/datasets
exp:
  num_candidate: 100
  k:
  - 1
  - 5
  - 10
  - 50
  - 100



In [3]:
import os
import sys
from pathlib import Path

import hydra
import numpy as np
import polars as pl
import torch
from hydra.core.hydra_config import HydraConfig
from omegaconf import DictConfig, OmegaConf
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm

import utils
import wandb
from utils.load import (
    load_image_embeddings,
    load_label_data,
    load_log_data,
    load_session_data,
)
from utils.metrics import calculate_metrics

In [4]:
image_embeddings_df = load_image_embeddings(
    Path(cfg.dir.data_dir), columns=["yad_no", "category", "emb_0"]
)

In [6]:
image_embeddings_df.head()

yad_no,category,emb_0
i64,str,f32
1,"""exterior""",-0.010736
1,"""exterior""",0.056188
1,"""exterior""",0.059149
1,"""facility""",-0.008016
1,"""facility""",0.010669


In [7]:
image_embeddings_df["category"].unique()

category
str
"""others"""
"""exterior"""
"""food"""
"""facility"""
"""room"""
